## ここで実施すること

既存LLMをファインチューニングすることで、柔軟かつ組織特色を帯びた対話能を実現する。

### 目標

- 質問に対して端的に回答する。

- 語尾に「ってな～。」とつける

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
import torch
import datasets
import random

#### モデルのロード

In [2]:
model_name = "rinna/japanese-gpt-1b"

## トークナイザー
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    legacy = False
)

## LLM
model = AutoModelForCausalLM.from_pretrained(model_name)

## 演算にコンピュータのGPUを利用する
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(44928, 2048)
    (wpe): Embedding(1024, 2048)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=6144, nx=2048)
          (c_proj): Conv1D(nf=2048, nx=2048)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=8192, nx=2048)
          (c_proj): Conv1D(nf=2048, nx=8192)
          (act): FastGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=2048, out_features=44928, bias=False)
)

In [3]:
def generate_sentence(
    input_messages_series: str,
    tkn,
    mdl
):
    ##
    input_tokens = tkn.encode(
        input_messages_series,
        return_tensors = 'pt',
        add_special_tokens = False
    ).to(mdl.device)
    ##
    output_tokens = mdl.generate(
        input_tokens,
        max_length = 100,
        min_length = 100,
        do_sample = True,
        top_k = 500,
        top_p = 0.95,
        pad_token_id = tkn.pad_token_id,
        bos_token_id = tkn.bos_token_id,
        eos_token_id = tkn.eos_token_id,
        bad_words_ids = [[tkn.unk_token_id]]
    )
    ##
    output_messages_series = tkn.decode(
        output_tokens[0],
        skip_special_tokens = True
    )
    
    ##
    return output_messages_series

In [4]:
%%time

print(generate_sentence(
    "random モジュールを使用するにはどうしたらいいですか？",
    tokenizer,
    model
))

random モジュールを使用するにはどうしたらいいですか?それは何ですか、何を変更する必要がありますか?エラーはどういう意味ですか? VBA 'random' を呼び出すには何を知っていますか? ‘random' 関数はインスタンス変数だけでなくプロセス変数も渡します。そして 'random' 関数は、インスタンス変数 'random' とプロセス変数 'RANDOMA' を組み合わせたもので す。プロセス変数にアクセスするために、 'random'
CPU times: user 1.71 s, sys: 192 ms, total: 1.9 s
Wall time: 1.61 s


#### データセットのロード

In [5]:
ds = datasets.load_dataset("cl-nagoya/auto-wiki-qa")
ds = ds["train"].shuffle().select(range(10000))
ds

Dataset({
    features: ['passage_id', 'query', 'answer', 'text', 'title', 'url'],
    num_rows: 10000
})

In [6]:
ds[0]

{'passage_id': 1934252,
 'query': '鶴舞商業高等学校と市原園芸高等学校が合併したのはいつ？',
 'answer': '2005年',
 'text': 'なお、鶴舞町消滅後の1955年(昭和30年)に鶴舞商業高等学校・市原園芸高等学校の2校に分離、両校は2005年に合併して千葉県立鶴舞桜が丘高等学校になるが、2019年に市原高等学校との統合により閉校している。',
 'title': '鶴舞町',
 'url': 'https://ja.wikipedia.org/wiki/%E9%B6%B4%E8%88%9E%E7%94%BA'}

In [7]:
def dialectize(example):
    ##
    r = random.randint(1, 3)
    if r == 1:
        tail = "ってな！"
    elif r == 2:
        tail = "ってな？？"
    else:
        tail = "ってな～"
        
    ##
    if example["answer"][-1] == "。":
        answer = example["answer"].replace("です。", tail).replace("ます。", tail)
    else:
        answer = example["answer"] + tail
        
    example["answer"] = answer
    return example

ds = ds.map(
    dialectize
)
ds[0]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'passage_id': 1934252,
 'query': '鶴舞商業高等学校と市原園芸高等学校が合併したのはいつ？',
 'answer': '2005年ってな～',
 'text': 'なお、鶴舞町消滅後の1955年(昭和30年)に鶴舞商業高等学校・市原園芸高等学校の2校に分離、両校は2005年に合併して千葉県立鶴舞桜が丘高等学校になるが、2019年に市原高等学校との統合により閉校している。',
 'title': '鶴舞町',
 'url': 'https://ja.wikipedia.org/wiki/%E9%B6%B4%E8%88%9E%E7%94%BA'}

In [8]:
MAX_LENGTH = 128

def serialize(example):
    example["tmp"] = "hhh:\n{0}\nqqq:\n{1}\naaa:\n{2}".format(example["text"], example["query"], example["answer"])
    return tokenizer(
        example["tmp"],
        max_length = MAX_LENGTH,
        padding = 'max_length',
        truncation = True
    )

ds = ds.map(
    serialize,
    remove_columns = ["query", "answer", "text", "title", "url"]
)
ds

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['passage_id', 'tmp', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [9]:
tokenizer.decode(ds[0]['input_ids'])

'hhh: なお、鶴舞町消滅後の1955年(昭和30年)に鶴舞商業高等学校・市原園芸高等学校の2校に分離、両校は2005年に合併して千葉県立鶴舞桜が丘高等学校になるが、2019年に市原高等学校との統合により閉校している。 qqq: 鶴舞商業高等学校と市原園芸高等学校が合併したのはいつ? aaa: 2005年ってな～</s>[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [10]:
ds = ds.shuffle().train_test_split(test_size = 0.1)
ds

DatasetDict({
    train: Dataset({
        features: ['passage_id', 'tmp', 'input_ids', 'attention_mask'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['passage_id', 'tmp', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [11]:
## チューニングパラメータ

num_train_epochs = 8
per_device_train_batch_size = 8
per_device_eval_batch_size = 4
learning_rate = 1e-4
weight_decay = 0.01

trained_model_name = 'seanayuuto/rinna-1b'

In [12]:
## チューニングの準備

training_args = TrainingArguments(
    output_dir = f"../{trained_model_name}",
    eval_strategy = 'epoch',
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = per_device_train_batch_size,
    per_device_eval_batch_size = per_device_eval_batch_size,
    learning_rate = learning_rate,
    weight_decay = weight_decay,
    logging_steps = 100
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = False
)

trainer = Trainer(
    model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = ds['train'],
    eval_dataset = ds['test']
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.842900,2.685695


In [ ]:
## huggingface保存

trainer.model.push_to_hub(
    private = False,
    repo_id = trained_model_name
)